# VGG16 from scratch for my dogscats classifier
This is possible from MOOC course www.fast.ai
Let's have a look at Vgg16 architecture.

In [1]:
import vgg16

Using TensorFlow backend.


In [2]:
from vgg16 import VGG16
vgg = VGG16()

In [3]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Lets Construct the same architecture

In [4]:
import keras
keras.__version__

'2.0.5'

In [5]:
import tensorflow
tensorflow.__version__

'1.2.1'

Vgg16 has:
1. 2 convolution layers with zero padding layer with maxpooling layer
2. 2 convolution layers with zero padding layer with maxpooling layer
3. 3 convolution layers with zero padding layer with maxpooling layer
4. 3 convolution layers with zero padding layer with maxpooling layer
5. 3 convolution layers with zero padding layer with maxpooling layer
6. With flattening layer

Vgg16 has 3 Dense layers. 
1. 1st 2 dense layer with 4096 neurons. 
2. 3rd dense layer cotains 1000 neurons (because it has to classify 1000 classes). 
3. It has also dropout layer with value of 0.5. 

# Dropout layer
Dropout is used to reduce overfitting. It deletes weigths of certian neurons ( a.k.a deactivating neurons ). Typically it's value is 50% (so 0.5). Some engineers use 0.25, 0.35, 0.45 respectively for each dense layer they add.  

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout
from keras.layers import Flatten

#This will be a sequential model
scratch = Sequential()

#1
scratch.add(Conv2D( 64,( 3, 3), input_shape = (224,224,3), activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(64,( 3, 3), activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2)) )

#2
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(128,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(128,( 3, 3),  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2)) )

#3
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(256,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(256,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(256,( 3, 3),  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2) ))

#3
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(512,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(512,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(512,( 3, 3),  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2)))

#3
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(512,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(512,( 3, 3),  activation='relu') )
scratch.add(ZeroPadding2D(padding=(1,1)))
scratch.add(Conv2D(512,( 3, 3),  activation='relu') )
scratch.add(MaxPooling2D(pool_size=(2,2) ))

#Flatten ing layer
scratch.add(Flatten())

#Dense 1
scratch.add(Dense(300,activation='relu'))
scratch.add(Dropout(0.5))    

#Dense 2
scratch.add(Dense(300,activation='relu'))
scratch.add(Dropout(0.5))    

#Finetuning
scratch.add(Dense(1,activation='sigmoid'))    

In [7]:
scratch.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()

train_generator = gen.flow_from_directory(
        'training_set',
        target_size=(224, 224),
        batch_size=3,
        class_mode='binary')

test_generator = gen.flow_from_directory(
        'test_set',
        target_size=(224, 224),
        batch_size=2,
        class_mode='binary')
#According to vgg16 model, target_size must be 224,224. I have given target_size as 64,64 to avoid errors. 
# I will figure it out and update. 
scratch.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=5,
        validation_data=test_generator,
        validation_steps=2)

Found 20 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/5
3/3 [==============================] - 20s - loss: 9.2935 - acc: 0.2222 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/5
3/3 [==============================] - 6s - loss: 3.5623 - acc: 0.7778 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/5
3/3 [==============================] - 9s - loss: 8.0188 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/5
3/3 [==============================] - 6s - loss: 10.6868 - acc: 0.3333 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/5
3/3 [==============================] - 5s - loss: 9.1739 - acc: 0.4271 - val_loss: 7.9712 - val_acc: 0.5000


## Saving the model. 

In [9]:
from keras.models import load_model
scratch.save('vgg_scratch.h5')

# NOTE: Supposed to be snippet code
```python
#Dense 1
scratch.add(Dense(4096,activation='relu'))
scratch.add(Dropout(0.5))    

#Dense 2
scratch.add(Dense(4096,activation='relu'))
scratch.add(Dropout(0.5))    

scratch.add(Dense(1,activation='sigmoid'))    
```
**been changed to** 
```python
#Dense 1
scratch.add(Dense(300,activation='relu'))
scratch.add(Dropout(0.5))    

#Dense 2
scratch.add(Dense(300,activation='relu'))
scratch.add(Dropout(0.5))    

scratch.add(Dense(1,activation='sigmoid'))    
```

#### Reason: 
* I ran this code on my local machine, which has low powered GPU. 
* This hardware limitations gave me "Resource Exhaustion Error"
* After googling and doing my research.
* I changed my 'massive' dense layers 
* I initially tested 1024, 700,512,400,350
* I realized that it can rum on max '300' 

* So, I finalized this and updated the code. 
* Accuracy is un imaginably low, It doesnt matter now, as this is completely for educational purposes. 
